Code for figuring out the object detection using open-cv

In [2]:
import cv2
import numpy as np
import time

In [3]:
def detect_puck_position(frame, lower_color, upper_color):
    if frame is None:
        print("Error: Invalid frame.")
        return None, None
    
    mask = cv2.inRange(frame, lower_color, upper_color)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # assume largest contour is the puck
        largest_contour = max(contours, key=cv2.contourArea)
        
        if cv2.contourArea(largest_contour) > 100:  # Filter out noise
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            center = (int(x), int(y))
            
            # Draw a point at the center of the puck
            cv2.circle(frame, center, 25, (0, 255, 0), -1)
            return center, frame
    
    return None, frame


In [12]:
def main():
    #black puck range
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([100, 100, 100])
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam. Try changing the device index.")
        return
    
    print("Starting puck detection. Press 'q' to quit.")

    
    fps_start_time = time.time()
    time_start = fps_start_time
    fps_frame_count = 0
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting...")
            break
        
        puck_position, processed_frame = detect_puck_position(frame, lower_color, upper_color)
        
        # Calculate and display FPS
        fps_frame_count += 1
        if (time.time() - fps_start_time) > 1:
            fps = fps_frame_count / (time.time() - fps_start_time)
            fps_frame_count = 0
            fps_start_time = time.time()
        
        # Display position information on the frame
        if puck_position:
            position_text = f"Position: {puck_position}, Time: {time.time() - time_start:.2f}"
            print(position_text)
            cv2.putText(processed_frame, position_text, (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(processed_frame, "No puck detected", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display the mask to help with debugging color thresholds
        mask = cv2.inRange(frame, lower_color, upper_color)
        cv2.imshow("Mask", mask)
            
        # Display the resulting frame
        cv2.imshow("Puck Detection", processed_frame)
        
        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # When everything is done, release the capture
    if cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == "__main__":
    main()


Starting puck detection. Press 'q' to quit.
Position: (484, 291), Time: 0.07
Position: (479, 296), Time: 0.15
Position: (481, 292), Time: 0.23
Position: (478, 304), Time: 0.30
Position: (484, 297), Time: 0.38
Position: (486, 286), Time: 0.47
Position: (481, 294), Time: 0.56
Position: (480, 293), Time: 0.62
Position: (482, 287), Time: 0.71
Position: (484, 291), Time: 0.79
Position: (483, 292), Time: 0.87
Position: (485, 285), Time: 0.95
Position: (486, 289), Time: 1.02
Position: (485, 288), Time: 1.11
Position: (486, 290), Time: 1.19
Position: (485, 287), Time: 1.27
Position: (486, 287), Time: 1.35
Position: (485, 286), Time: 1.43
Position: (485, 285), Time: 1.51
Position: (485, 285), Time: 1.59
Position: (481, 293), Time: 1.67
Position: (485, 285), Time: 1.75
Position: (486, 282), Time: 1.83
Position: (483, 290), Time: 1.91
Position: (486, 285), Time: 1.99
Position: (484, 284), Time: 2.07
Position: (485, 284), Time: 2.15
Position: (488, 280), Time: 2.23
Position: (485, 285), Time: 2.31